# 分割表を扱う

ここではクロス集計でも現れた分割表の分析を扱います．
別資料（仮説と確率分布の違いDSL_Hypothesis）でも扱ったように，仮説の設定により，同じデータでも導かれる方向は変わってきます．
ここでは考えたい分析事例に次のようなものあがあります．

| |肺がんあり|肺がんなし|
|-|-|-|
|喫煙あり| 231| 23036|
|喫煙なし|26 |10813|

このとき喫煙のありなしで比率を比べてみましょう．



In [20]:
# データを変数に代入
Cancer <- c(231,26,23036,10813)
# 1,3番目が喫煙あり
# 2,4番目が喫煙なし
Cancer[1]/Cancer[3]
Cancer[2]/Cancer[4]

[1] 0.01002778

[1] 0.002404513

実行プログラムを書く人が何番目が．．．と判断するのは間違いやすいものです．matrixという関数により表と同じく正方形状に配置して扱えます．


In [21]:
CancerMat <- matrix(Cancer, 2)
CancerMat
# 1列目を表示
CancerMat[,1]
# 2列目を表示
CancerMat[1,]

231,23036
26,10813


[1] 231  26

[1]   231 23036

両者の比率を比較することを考えたとき，いろいろな方法が考えられます．
いくつかあげます．

1. 喫煙ありなしを含めた肺がんありの比率（喫煙ありとなしの重み付き平均と一致）をもとに対して，喫煙ありなしの比が大きいとか小さいとかいう
1. 喫煙ありにおける比に対して喫煙なしにおける比を調べてそれが大きいとか小さいとかいう
1. 喫煙なしにおける比に対して喫煙ありにおける比を調べてそれが大きいとか小さいとかいう

これらについて，比が大きいとか小さいとかの判断を確率（p値）によって検討できるRの関数にPpro.testがあります．
以下はその実行例です．

In [22]:
# 喫煙なし全体の度数
Ari <- sum(CancerMat[,1])
# 喫煙あり全体の度数
Nashi <- sum(CancerMat[,2])
# 全体における肺がんなしに対するありの比
RatioAll <- Ari/Nashi
RatioAll
# 1の場合
# 喫煙ありの全体に対する比
prop.test(CancerMat[1,1], Ari, p=RatioAll)
# 喫煙なしの全体に対する比
prop.test(CancerMat[2,1],Nashi, p=RatioAll)
# 2の場合，つまり喫煙なしに対する喫煙ありの比較
prop.test(CancerMat[1,1], Ari, p=CancerMat[2,1]/Nashi)
# 3の場合，つまり喫煙ありに対する喫煙ありの比較
prop.test(CancerMat[2,1], Nashi, p=CancerMat[1,1]/Ari)

[1] 0.007592543

Warning message in prop.test(CancerMat[1, 1], Ari, p = RatioAll):
“Chi-squared approximation may be incorrect”



	1-sample proportions test with continuity correction

data:  CancerMat[1, 1] out of Ari, null probability RatioAll
X-squared = 26974, df = 1, p-value < 2.2e-16
alternative hypothesis: true p is not equal to 0.007592543
95 percent confidence interval:
 0.8536771 0.9316333
sample estimates:
        p 
0.8988327 



	1-sample proportions test with continuity correction

data:  CancerMat[2, 1] out of Nashi, null probability RatioAll
X-squared = 208.31, df = 1, p-value < 2.2e-16
alternative hypothesis: true p is not equal to 0.007592543
95 percent confidence interval:
 0.000512288 0.001142816
sample estimates:
           p 
0.0007681172 


Warning message in prop.test(CancerMat[1, 1], Ari, p = CancerMat[2, 1]/Nashi):
“Chi-squared approximation may be incorrect”



	1-sample proportions test with continuity correction

data:  CancerMat[1, 1] out of Ari, null probability CancerMat[2, 1]/Nashi
X-squared = 268888, df = 1, p-value < 2.2e-16
alternative hypothesis: true p is not equal to 0.0007681172
95 percent confidence interval:
 0.8536771 0.9316333
sample estimates:
        p 
0.8988327 



	1-sample proportions test with continuity correction

data:  CancerMat[2, 1] out of Nashi, null probability CancerMat[1, 1]/Ari
X-squared = 300212, df = 1, p-value < 2.2e-16
alternative hypothesis: true p is not equal to 0.8988327
95 percent confidence interval:
 0.000512288 0.001142816
sample estimates:
           p 
0.0007681172 


事前準備やどの要素を取り出すとか少しめんどくさい気もします．
もっと簡単な分析方法はないのでしょうか．
上にあげた表の形式を分割表といいます．
それに対して確率を求める関数に$\chi^2$（カイ自乗）検定関数chisq.testがあります．

以下では，その実行に必要な分だけ再掲して実行しています．

In [23]:
Cancer <- c(231,26,23036,10813)
CancerMat <- matrix(Cancer, 2)
chisq.test(CancerMat)


	Pearson's Chi-squared test with Yates' continuity correction

data:  CancerMat
X-squared = 55.052, df = 1, p-value = 1.174e-13


平均値を扱った以前の例題と違って，計数データの処理では信頼区間を出力していないことに注意してください．
確率（p値）は非常に小さいのですが，上で比べた場合よりもさらに小さくなっているようです．

以下ではこの検定関数の背景を簡単な例を通じて調べます．

## 分割表に対するp値
参考：奥村晴彦「Rで楽しむ統計」，5.2節

ある方策に男女5人ずつから意見を聞いたところ以下のようになりました．

| |賛成|反対|
|-|-|-|
|女| 3| 2|
|男|1 |4|

この問題は赤玉4個と白玉6個が入っている箱から5個取り出してみたとする，確率の問題と同じです．

| |赤玉|白玉|
|-|-|-|
|取り出した| 3| 2|
|箱の中|1 |4|

この個数の組み合わせが生じる確率は
$$
\frac{{}_4 C_3\cdot {}_6 C_2}{{}_{10} C_5}
$$
であり，Rの組み合わせの数を求める関数chooseを使って次のように計算できます．

In [24]:
choose(4,3)*choose(6,2)/choose(10,5)

[1] 0.2380952

赤玉の個数は4個なので，他の個数の出方も一通り計算できます．

In [25]:
for (i in 0:4) {
  print(choose(4,i)*choose(6,5-i)/choose(10,5))
}

[1] 0.02380952
[1] 0.2380952
[1] 0.4761905
[1] 0.2380952
[1] 0.02380952


この一続きの計算に対応する確率を計算するfisher.test関数があります．

In [26]:
Deme <- c(3,1,2,4)
DemeMat <- matrix(Deme,2)
fisher.test(DemeMat)


	Fisher's Exact Test for Count Data

data:  DemeMat
p-value = 0.5238
alternative hypothesis: true odds ratio is not equal to 1
95 percent confidence interval:
   0.218046 390.562917
sample estimates:
odds ratio 
  4.918388 


この計算結果0.5238は上の確率のどこからどこまでを足し合わせたものでしょうか．
実は最初の目の出方を除いた確率合計なのです．
確かめてみます．

In [27]:
s <- 0
for (i in 0:4) {
# 最初の個数の状態を除く
  if (i != 2) {
    x <- choose(4,i)*choose(6,5-i)/choose(10,5)
    s <- s+x
   print(s)
  }
}

[1] 0.02380952
[1] 0.2619048
[1] 0.5
[1] 0.5238095


関数fisher.testは正確確率と呼ばれるものを計算しています．
$\chi^2$検定は，この正確確率の近似値を計算します．
なお，この関数の出力の末尾にあるオッズ比（odds ratio）は治験でも多く使われる情報ですが解説は行いません．
興味のある人は勉強してみてください．